https://coursera.cs.princeton.edu/algs4/assignments/8puzzle/specification.php

In [3]:
struct Board{
    var tiles: [Int]
    var dim: Int
    var solved: [Int]
    var board2D: [[Int]]
    init(tiles: [[Int]]){
        self.dim = tiles[0].count
        self.tiles = Array(0...self.dim*self.dim-1)
        self.solved = Array(1...self.dim*self.dim-1)
        self.solved.append(0)
        self.board2D = tiles
        for (row_idx, row) in tiles.enumerated(){
            for (col_idx, val) in row.enumerated(){
                self.tiles[row_idx*self.dim + col_idx] = val
            }
        }
    }
    func toString()->String{
        var repr = "\(self.dim)"
        for (idx, val) in self.tiles.enumerated(){
            if idx%self.dim == 0{
                repr += "\n"
            }
            repr += " \(val)"
        }
        return repr
    }
    func dimension()->Int{
        return self.dim
    }
    func hamming()->Int{
        var dist = 0
        for idx in 0...self.tiles.count-2{
            if self.tiles[idx] != self.solved[idx]{
                dist+=1
            }
        }
        return dist
    }
    func manhattan()->Int{
        var dist = 0
        for idx in 0...self.tiles.count-1{
            if (self.tiles[idx] != idx+1) && (self.tiles[idx] != 0) {
                    dist += self.manh_single(self.tiles[idx]-1, idx)
            }
        }
        return dist
    }
    private func manh_single(_ idx: Int,_ idx2: Int)->Int{
        let v1 = self.idTo2D(idx)
        let v2 = self.idTo2D(idx2)
        return abs(v2.0-v1.0)+abs(v2.1-v1.1)
    }
    private func idTo2D(_ idx: Int)->(Int, Int){
        let col = idx % self.dim
        let row = (idx - col)/self.dim
        return (row, col)
    }
    private func idFrom2D(_ val: (Int, Int))->Int{
        return val.0*3+val.1
    }
    func isGoal()->Bool{
        self.tiles == self.solved
    }
    func neighbors()->[[[Int]]]{
        var boards = [[[Int]]]()
        let empty_idx = self.tiles.firstIndex(of: 0)!
        let empty_idx_2d = self.idTo2D(empty_idx)
        if empty_idx_2d.0 != 0{
            var top_pos = empty_idx_2d
            top_pos.0 -= 1
            var neighbor = self.tiles
            neighbor.swapAt(self.idFrom2D(top_pos), empty_idx)
            boards.append(self.tilesTo2D(neighbor))
        }
        if empty_idx_2d.0 != self.dim-1{
            var bottom_pos = empty_idx_2d
            bottom_pos.0 += 1
            var neighbor = self.tiles
            neighbor.swapAt(self.idFrom2D(bottom_pos), empty_idx)
            boards.append(self.tilesTo2D(neighbor))
        }
        if empty_idx_2d.1 != 0{
            var left_pos = empty_idx_2d
            left_pos.1 -= 1
            var neighbor = self.tiles
            neighbor.swapAt(self.idFrom2D(left_pos), empty_idx)
            boards.append(self.tilesTo2D(neighbor))
        }
        if empty_idx_2d.1 != self.dim-1{
            var right_pos = empty_idx_2d
            right_pos.1 += 1
            var neighbor = self.tiles
            neighbor.swapAt(self.idFrom2D(right_pos), empty_idx)
            boards.append(self.tilesTo2D(neighbor))
        }
        return boards
    }
    func tilesTo2D(_ tiles: [Int])->[[Int]]{
        var board2d = self.board2D
        for (row_idx, row) in board2d.enumerated(){
            for (col_idx, _) in row.enumerated(){
                board2d[row_idx][col_idx] = tiles[row_idx*self.dim + col_idx]
            }
        }
        return board2d
    }
    func twin()->[[Int]]{
        var tiles=self.tiles
        let idx1 = self.tiles.firstIndex(of: 1)!
        let idx2 = self.tiles.firstIndex(of: 2)!
        tiles.swapAt(idx1, idx2)
        return self.tilesTo2D(tiles)
    }
}
extension Board: Equatable {
    static func ==(lhs: Board, rhs: Board) -> Bool {
        return lhs.tiles == rhs.tiles
    }
}
var board = Board(tiles:[[8, 1, 3], [4, 0, 2], [7, 6, 5]])
assert("3\n 8 1 3\n 4 0 2\n 7 6 5"==board.toString(), "toString")
assert(5 == board.hamming(), "hamming")
assert(10 == board.manhattan(), "manhattan")
assert(false==board.isGoal(), "goal")
assert([[[8, 0, 3], [4, 1, 2], [7, 6, 5]], 
        [[8, 1, 3], [4, 6, 2], [7, 0, 5]], 
        [[8, 1, 3], [0, 4, 2], [7, 6, 5]], 
        [[8, 1, 3], [4, 2, 0], [7, 6, 5]]] == board.neighbors(), "neighbors")

In [4]:
struct MinPQ{
    var pq: [Int?] = []
    var n: Int = 0
    func isEmpty()->Bool{
        return self.n == 0
    }
    func size()->Int{
        return self.n
    }
    func min()->Int{
        return self.pq[0]!
    }
    mutating func insert(_ el: Int){
        self.pq.append(el)
        let el_idx = self.pq.firstIndex(of: el)!
        self.swim(el_idx)
    }
    mutating func delMin()->Int {
        let first = self.pq.removeFirst()!
        self.sink(0)
        return first
    }
    mutating func sink(_ idx: Int){
        var idx = idx
        while (2*idx <= self.n) {
            var j = 2*idx
            if (j < self.n && self.greater(j, j+1)){
                j += 1
            }
            if (!greater(idx, j)) {
                break
            }
            self.pq.swapAt(idx, j)
            idx = j
        }
    }
    mutating private func swim(_ idx: Int){
        var idx = idx
        while (idx > 0 && self.greater(idx/2, idx)) {
            self.pq.swapAt(idx, idx/2)
            idx = idx/2
        }
    }
    private func greater(_ first: Int, _ second: Int)->Bool{
        return self.pq[first]! > self.pq[second]!
    }
}
var pq = MinPQ()